In [6]:
import pandas as pd

In [7]:
# Assuming your files are named file1.xlsx, file2.xlsx, and file3.xlsx
file_paths = ['...']

# Create an empty DataFrame to store the concatenated data
Ah = pd.DataFrame()

# Iterate through each file and extract the last sheet
for file_path in file_paths:
    df = pd.read_excel(file_path, sheet_name=-1)  # -1 refers to the last sheet
    
    # Ensure the column ends at a multiple of 10
    remainder = len(df) % 10
    
    if remainder != 0:
        additional_rows = 10 - remainder
        last_value = df['Discharge_Capacity(Ah)'].iloc[-1]
        additional_data = pd.DataFrame({'Discharge_Capacity(Ah)': [last_value] * additional_rows})
        df = pd.concat([df, additional_data], ignore_index=True)
    Ah = pd.concat([Ah, df['Discharge_Capacity(Ah)']], axis=0, ignore_index=True)
# Rename the column to 'Discharge_Capacity(Ah)'
Ah = Ah.rename(columns={0: 'Discharge_Capacity(Ah)'})

In [8]:
def soh_percent(Ah):
    soh = []
    soh_perc = []

    # Convert the 'Discharge_Capacity(Ah)' column to numeric
    #Ah['Discharge_Capacity(Ah)'] = pd.to_numeric(Ah['Discharge_Capacity(Ah)'], errors='coerce')

    for i in range(0, len(Ah), 10):
        soh.append(Ah['Discharge_Capacity(Ah)'].iloc[i])

    for i in soh:
        aux = ((soh[0] - i) / soh[0]) * 100
        soh_perc.append(aux)

    return soh_perc

def ah_throughput(Ah):
    add_num = 0
    Ah_cycle = []
    
    # Convert the 'Discharge_Capacity(Ah)' column to numeric
    Ah['Discharge_Capacity(Ah)'] = pd.to_numeric(Ah['Discharge_Capacity(Ah)'], errors='coerce')

    for i in range(1, len(Ah), 10):
        add_num += Ah['Discharge_Capacity(Ah)'].iloc[i:i+9].sum()
        Ah_cycle.append(2 * add_num)

    return Ah_cycle

In [9]:
Q_loss = soh_percent(Ah)
Ah_thr = ah_throughput(Ah)

In [10]:
# Data frame
df = pd.DataFrame({'Var1': Ah_thr, 'Var2': Q_loss})
df.to_excel('data_clean_LFP_16.xlsx', index=False)